In [ ]:
# Misc
import random
import numpy as np
import pandas as pd
import tqdm
from tqdm import tqdm
from tqdm import tqdm_notebook
import math
import os
import time
import sys
from torch import tensor

In [ ]:
# Plotting
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Sklearn
import sklearn
from sklearn.preprocessing import MinMaxScaler
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
# Pytorch
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
# Keras 
import keras

In [ ]:
# Helpers
# from Adverse import lowProFool, deepfool
# from Metrics import *
# Misc
import numpy as np

# Pytorch
import torch
import torch.nn as nn
from torch.autograd import Variable
# from torch.autograd.gradcheck import zero_gradients
def zero_gradients(x):
  if x.grad is not None:
      x.grad.zero_()

import numpy as np
import pandas as pd
import torch
import sklearn
from sklearn.neighbors import NearestNeighbors


def get_metrics(config, list_metrics, n_neighbors=2):
    """
    Generates an adversarial examples x' from an original sample x. Expected to contain
            Dataset, MaxIters, Alpha, Lambda, TrainData, TestData, ValidData, Scaler,
            FeatureNames, Target, Weights, Bounds, Model
    :param config: dictionnary containing the configuration for the experiment
    :param list_metrics: dictionnary containing the metrics to be computed. Choose from
            from SuccessRate, iter_means, iter_std, normdelta_median, normdelta_mean,
            n_std, weighted_median, weighted_mean, w_std, mean_dists_at_org,
            median_dists_at_org, mean_dists_at_tgt, mean_dists_at_org_weighted, mdow_std,
            median_dists_at_org_weighted, mean_dists_at_tgt_weighted, mdtw_std, prop_same_class_arg_org,
            prop_same_class_arg_adv,
    :param n_neighbors: number of neighbors to compute the distance to n_neighbors closest neighbors
    """

    metrics_for_conf = []
    df_test = config['TestData']
    dfs_adv = config['AdvData']
    
    for method, df_adv in dfs_adv.items():
        metrics_for_method = [method]
        # Get success rate before removing samples from dataframe
        if list_metrics['SuccessRate']:
            sr = metric_success_rate_for(df_adv)
            metrics_for_method.append(sr)

        # Removes samples that did cross frontier
        df_adv = remove_non_converted(df_adv)        
        df_adv = add_normdelta_to(df_adv, config, df_test)

        # Adding proportion of neighbors from diff classes
        df_adv, df_adv_weighted = add_maj_neighbors(df_adv, df_test, config, n_neighbors=n_neighbors)            

        # Mean, std, number of iterations
        if list_metrics['iter_means']:
            means_iters, stds_iters = mean_norm_for_col(df_adv, col='iters')
            metrics_for_method.append(means_iters)
            if list_metrics['iter_std']:
                metrics_for_method.append(stds_iters)

        # Median, norm of perturbation
        if list_metrics['normdelta_median']:
            median = median_norm_for_col(df_adv, col='normdelta')
            metrics_for_method.append(median)

        # Mean, std, norm of perturbation
        if list_metrics['normdelta_mean']:
            means, stds = mean_norm_for_col(df_adv, col='normdelta')
            metrics_for_method.append(means)
            if list_metrics['n_std']:
                metrics_for_method.append(stds)

        # Median, norm of perturbation, weighted
        if list_metrics['weighted_median']:
            median_w = median_norm_for_col(df_adv, col='normdelta_weighted')
            metrics_for_method.append(median_w)

        # Mean, std, norm of perturbation, weighted
        if list_metrics['weighted_mean']:
            means_w, stds_w = mean_norm_for_col(df_adv, col='normdelta_weighted')
            metrics_for_method.append(means_w)
            if list_metrics['w_std']:
                metrics_for_method.append(stds_w) 

        # Mean, std, number of neighbors of a particular class at perturbed sample
        if list_metrics['mean_dists_at_org']:
            mean, std = mean_norm_for_col(df_adv, col='mean_dists_at_org')
            metrics_for_method.append(mean)

        # Mean, std, number of neighbors of a particular class at perturbed sample
        if list_metrics['median_dists_at_org']:
            med = median_norm_for_col(df_adv, col='mean_dists_at_org')
            metrics_for_method.append(med)

        # Mean, std, number of neighbors of a particular class at perturbed sample
        if list_metrics['mean_dists_at_tgt']:
            mean, std = mean_norm_for_col(df_adv, col='mean_dists_at_tgt')
            metrics_for_method.append(mean)

        # Mean, std, number of neighbors of a particular class at perturbed sample
        if list_metrics['mean_dists_at_org_weighted']:
            mean, std = mean_norm_for_col(df_adv_weighted, col='mean_dists_at_org')
            metrics_for_method.append(mean)
            if list_metrics['mdow_std']:
                metrics_for_method.append(std)

        # Mean, std, number of neighbors of a particular class at perturbed sample
        if list_metrics['median_dists_at_org_weighted']:
            median = median_norm_for_col(df_adv_weighted, col='mean_dists_at_org')
            metrics_for_method.append(median)

        # Mean, std, number of neighbors of a particular class at perturbed sample
        if list_metrics['mean_dists_at_tgt_weighted']:
            mean, std = mean_norm_for_col(df_adv_weighted, col='mean_dists_at_tgt')
            metrics_for_method.append(mean)
            if list_metrics['mdtw_std']:
                metrics_for_method.append(std)

        # Mean, std, number of neighbors of a particular class at perturbed sample
        if list_metrics['prop_same_class_arg_org']:
            mean, std = mean_norm_for_col(df_adv, col='prop_same_class_arg_org')
            metrics_for_method.append(mean)

        # Mean, std, number of neighbors of a particular class at perturbed sample
        if list_metrics['prop_same_class_arg_adv']:
            mean, std = mean_norm_for_col(df_adv, col='prop_same_class_arg_adv')
            metrics_for_method.append(mean)
            
        metrics_for_conf.append(metrics_for_method)
    return metrics_for_conf
  
def metric_success_rate_for(df):
    return len(df[df['orig_pred'] != df['adv_pred']]) / df.shape[0]

def remove_non_converted(df):
    df_return = df.copy()
    return df[df['orig_pred'] != df['adv_pred']]

def mean_norm_for_col(df, col):
    tmp = df[col]    
    mean, std = np.mean(tmp), np.std(tmp)
    return (mean, std)

def median_norm_for_col(df, col):
    tmp = df[col]    
    median = np.median(tmp)
    return median

def add_normdelta_to(df_adv, conf, df):
    # Drop columns if already there
    df_return = df_adv.copy()
    if 'normdelta' in df_return.columns:
        df_return = df_return.drop(columns='normdelta')
    if 'normdelta_weighted' in df_return.columns:
        df_return = df_return.drop(columns='normdelta_weighted')
        
    feature_names = conf['FeatureNames']
    weights = conf['Weights']

    norms = []
    norms_weighted = []
    
    # Iterate over all rows
    for index, row in df_return.iterrows():
        orig = df.loc[index][feature_names].values
        adv = row[feature_names].values 
        
        # Compute deltas
        delta = np.abs(orig-adv)
        assert(len(delta) == len(weights))
        
        # Norms delta
        norms.append(np.linalg.norm(delta))
        
        # Norms delta weighted
        norms_weighted.append(np.linalg.norm(delta * weights))

    df_return.insert(0, 'normdelta', norms)
    df_return.insert(0, 'normdelta_weighted', norms_weighted)
    
    return df_return


def get_majority_neighbors(df_adv, df_orig, conf, knn, n_neighbors):
    
    # orig, adv
    mean_dists = [[], []]
    prop_same_class = [[], []]
    
    feature_names = conf['FeatureNames']
    target = conf['Target']    
    
    # For each sample
    for index, row in df_adv.iterrows():
        
        orig = df_orig.loc[index][feature_names].values
        adv = row[feature_names].values
        
        preds = [row['orig_pred'], row['adv_pred']]
        samples = [orig, adv]
        
        for i in range(len(preds)):
            
            sample = samples[i]
            pred = preds[i]
            
            distance, neighbors_idxs = knn.kneighbors([sample], n_neighbors)
            neighbors_samples = df_orig.iloc[neighbors_idxs[0]]
            
           
            distance = [distance[0][1:]]
            neighbors_idxs = [neighbors_idxs[0][1:]]
            

            # Distance to closest neighbors
            if len(distance[0]) > 0 :
                dst_mean = np.mean(distance[0])
            else:
                print('Error, no neighbor found')
            mean_dists[i].append(dst_mean)
            
            neighbors_pts_target = np.array(neighbors_samples[target]).astype(int)
            prop = list(neighbors_pts_target).count(pred)
            prop_same_class[i].append(float(prop)/float(n_neighbors))
                        
    return mean_dists, prop_same_class

def add_maj_neighbors_to(df_adv, df_orig, conf, knn, n_neighbors):
    df_return = df_adv.copy()
    
    if 'mean_dists_at_org' in df_return.columns:
        df_return = df_return.drop(columns='mean_dists_at_org')
    if 'mean_dists_at_tgt' in df_return.columns:
        df_return = df_return.drop(columns='mean_dists_at_tgt')
    if 'prop_same_class_arg_org' in df_return.columns:
        df_return = df_return.drop(columns='prop_same_class_arg_org')
    if 'prop_same_class_arg_adv' in df_return.columns:
        df_return = df_return.drop(columns='prop_same_class_arg_adv')
        
    mean_dists, prop_same_class = get_majority_neighbors(df_adv, df_orig, conf, knn, n_neighbors)
    
    df_return.insert(0, 'mean_dists_at_org', mean_dists[0])
    df_return.insert(0, 'mean_dists_at_tgt', mean_dists[1])

    df_return.insert(0, 'prop_same_class_arg_org', prop_same_class[0])
    df_return.insert(0, 'prop_same_class_arg_adv', prop_same_class[1])
    
    return df_return

def scale_data(conf, df_orig):
    print('Before')
    print(df.describe(include='all'))
    print(weights)
    for col, weight in zip(list(df.columns), weights):
        df[col] = df[col].apply(lambda x: x * weight)
        
    bounds = [[bounds[i][x] * weight for x, weight in enumerate(weights)] for i in range(len(bounds))]
    print(df.describe(include='all'))
    return df, bounds

def weighted_distance(x, y, w):
    sum_ = 0
    assert(len(x) == len(y) == len(w))
    for i in range(len(x)):
        sum_ += (w[i] * (y[i] - x[i])) ** 2
    sum_ = np.sqrt(sum_)
    return sum_

def add_maj_neighbors(df_adv, df_orig, conf, n_neighbors):
    # Otherwise we have issues because the KNN returns indexes in len(df) and not based on the real indexes on the samples
    df_adv = df_adv.reset_index().drop(columns=['index'])
    df_orig = df_orig.reset_index().drop(columns=['index'])
    weights = conf['Weights']

    assert(weights[0] > 0)

    feature_names = conf['FeatureNames']
    target = conf['Target']
        
        
    knn = NearestNeighbors(n_neighbors=n_neighbors, metric='l2')
    knn.fit(df_orig[feature_names], df_orig[target])
    
    knn_weighted = NearestNeighbors(n_neighbors=n_neighbors, metric=weighted_distance, metric_params={'w' : weights})
    knn_weighted.fit(df_orig[feature_names], df_orig[target])

    
    df_adv_return = add_maj_neighbors_to(df_adv, df_orig, conf, knn, n_neighbors)
    df_adv_weighted = add_maj_neighbors_to(df_adv, df_orig, conf, knn_weighted, n_neighbors)
    
    return df_adv_return, df_adv_weighted


In [ ]:
# Misc
import numpy as np

# Pytorch
import torch
import torch.nn as nn
from torch.autograd import Variable
# from torch.autograd.gradcheck import zero_gradients

# Clipping function
def clip(current, low_bound, up_bound):
    assert(len(current) == len(up_bound) and len(low_bound) == len(up_bound))
    low_bound = torch.FloatTensor(low_bound)
    up_bound = torch.FloatTensor(up_bound)
    clipped = torch.max(torch.min(current, up_bound), low_bound)
    return clipped


def lowProFool(x, model, weights, bounds, maxiters, alpha, lambda_, cat_indices=[]):
    """
    Generates an adversarial examples x' from an original sample x

    :param x: tabular sample
    :param model: neural network
    :param weights: feature importance vector associated with the dataset at hand
    :param bounds: bounds of the datasets with respect to each feature
    :param maxiters: maximum number of iterations ran to generate the adversarial examples
    :param alpha: scaling factor used to control the growth of the perturbation
    :param lambda_: trade off factor between fooling the classifier and generating imperceptible adversarial example
    :return: original label prediction, final label prediction, adversarial examples x', iteration at which the class changed
    """

    r = Variable(torch.FloatTensor(1e-4 * np.ones(x.numpy().shape)), requires_grad=True) 
    v = torch.FloatTensor(np.array(weights))
    
    output = model.forward(x + r)
    orig_pred = output.max(0, keepdim=True)[1].cpu().numpy()
    target_pred = np.abs(1 - orig_pred)
    
    target = [0., 1.] if target_pred == 1 else [1., 0.]
    target = Variable(torch.tensor(target, requires_grad=False)) 
    
    lambda_ = torch.tensor([lambda_])
    
    bce = nn.BCELoss()
    l1 = lambda v, r: torch.sum(torch.abs(v * r)) #L1 norm
    l2 = lambda v, r: torch.sqrt(torch.sum(torch.mul(v * r,v * r))) #L2 norm
    
    best_norm_weighted = np.inf
    best_pert_x = x
    
    loop_i, loop_change_class = 0, 0
    while loop_i < maxiters:
            
        zero_gradients(r)

        # Computing loss 
        loss_1 = bce(output, target)
        loss_2 = l2(v, r)
        loss = (loss_1 + lambda_ * loss_2)

        # Get the gradient
        loss.backward(retain_graph=True)
        grad_r = r.grad.data.cpu().numpy().copy()
        
        # Guide perturbation to the negative of the gradient
        ri = - grad_r
    
        # limit huge step
        ri *= alpha

        # Adds new perturbation to total perturbation
        r = r.clone().detach().cpu().numpy() + ri
        
        # For later computation
        r_norm_weighted = np.sum(np.abs(r * weights))
        
        # Ready to feed the model
        r = Variable(torch.FloatTensor(r), requires_grad=True) 
        
        # Compute adversarial example
        xprime = x + r
        
        # Clip to stay in legitimate bounds
        xprime = clip(xprime, bounds[0], bounds[1])
        
        # Classify adversarial example
        output = model.forward(xprime)
        output_pred = output.max(0, keepdim=True)[1].cpu().numpy()
        
        # Keep the best adverse at each iterations
        if output_pred != orig_pred and r_norm_weighted < best_norm_weighted:
            best_norm_weighted = r_norm_weighted
            best_pert_x = xprime

        if output_pred == orig_pred:
            loop_change_class += 1
            
        loop_i += 1 
        
    # Clip at the end no matter what
    best_pert_x = clip(best_pert_x, bounds[0], bounds[1])
    output = model.forward(best_pert_x)
    output_pred = output.max(0, keepdim=True)[1].cpu().numpy()

    return orig_pred, output_pred, best_pert_x.clone().detach().cpu().numpy(), loop_change_class 

# Forked from https://github.com/LTS4/DeepFool
def deepfool(x_old, net, maxiters, alpha, bounds, weights=[], overshoot=0.002):
    """
    :param image: tabular sample
    :param net: network 
    :param maxiters: maximum number of iterations ran to generate the adversarial examples
    :param alpha: scaling factor used to control the growth of the perturbation
    :param bounds: bounds of the datasets with respect to each feature
    :param weights: feature importance vector associated with the dataset at hand
    :param overshoot: used as a termination criterion to prevent vanishing updates (default = 0.02).
    :return: minimal perturbation that fools the classifier, number of iterations that it required, new estimated_label and perturbed image
    """
    
    input_shape = x_old.numpy().shape
    x = x_old.clone()
    x = Variable(x, requires_grad=True)
    
    output = net.forward(x)
    orig_pred = output.max(0, keepdim=True)[1] # get the index of the max log-probability

    origin = Variable(torch.tensor([orig_pred], requires_grad=False))

    I = []
    if orig_pred == 0:
        I = [0, 1]
    else:
        I = [1, 0]
        
    w = np.zeros(input_shape)
    r_tot = np.zeros(input_shape)
    
    k_i = origin
 
    loop_i = 0
    while torch.eq(k_i, origin) and loop_i < maxiters:
                
        # Origin class
        output[I[0]].backward(retain_graph=True)
        grad_orig = x.grad.data.numpy().copy()
        
        # Target class
        zero_gradients(x)
        output[I[1]].backward(retain_graph=True)
        cur_grad = x.grad.data.numpy().copy()
            
        # set new w and new f
        w = cur_grad - grad_orig
        f = (output[I[1]] - output[I[0]]).data.numpy()

        pert = abs(f)/np.linalg.norm(w.flatten())
    
        # compute r_i and r_tot
        # Added 1e-4 for numerical stability
        r_i =  (pert+1e-4) * w / np.linalg.norm(w)   
        
        if len(weights) > 0:
            r_i /= np.array(weights)

        # limit huge step
        r_i = alpha * r_i / np.linalg.norm(r_i) 
            
        r_tot = np.float32(r_tot + r_i)
        
        
        pert_x = x_old + (1 + overshoot) * torch.from_numpy(r_tot)

        if len(bounds) > 0:
            pert_x = clip(pert_x, bounds[0], bounds[1])
                
        x = Variable(pert_x, requires_grad=True)
 
        output = net.forward(x)
        
        k_i = torch.tensor(np.argmax(output.data.cpu().numpy().flatten()))
                    
        loop_i += 1

    r_tot = (1+overshoot)*r_tot    
    pert_x = clip(pert_x, bounds[0], bounds[1])

    return orig_pred, k_i, pert_x.cpu(), loop_i

In [ ]:
SEED = 0
DATASET = 'Transactions'

In [ ]:
# def get_df(dataset):
#     assert(dataset == 'credit-g')
    
#     dataset = fetch_openml(dataset)
#     target = 'target'
#     df = pd.DataFrame(data= np.c_[dataset['data'], dataset[target]], columns= dataset['feature_names'] + [target])  

#     # Renaming target for training later
#     df[target] = df[target].apply(lambda x: 0.0 if x == 'bad' or x == 0.0 else 1.0)

#     # Subsetting features to keep only continuous, discrete and ordered categorical
#     feature_names = ['checking_status', 'duration', 'credit_amount',
#                  'savings_status','employment','installment_commitment',
#                  'residence_since','age','existing_credits','num_dependents',
#                  'own_telephone','foreign_worker']

#     df = df[feature_names + [target]]

#     # Casting to float for later purpose
#     df = df.astype(float)
#     return df, target, feature_names
#def get_df(dataset):
 #   assert(dataset == 'credit-g')
    
  #  dataset = fetch_openml(dataset)
   # target = 'target'
    #df = pd.DataFrame(data= np.c_[dataset['data'], dataset[target]], columns= dataset['feature_names'] + [target])  

    # Renaming target for training later
    #df[target] = df[target].apply(lambda x: 0.0 if x == 'bad' or x == 0.0 else 1.0)

    # Subsetting features to keep only continuous, discrete and ordered categorical
    #feature_names = ['checking_status', 'duration', 'credit_amount',
     #            'savings_status','employment','installment_commitment',
      #           'residence_since','age','existing_credits','num_dependents',
       #          'own_telephone','foreign_worker']

    #df = df[feature_names + [target]]
    #df['checking_status']=df['checking_status'].map({'<0':0, '0<=X<200':2, 'no checking':1, '>=200':3})
    #df['savings_status']=df['savings_status'].map({'no known savings':0, '<100':1, '500<=X<1000':3, '>=1000':4, '100<=X<500':2})
    #df['employment']=df['employment'].map({'>=7':4, '1<=X<4':2, '4<=X<7':3, 'unemployed':0, '<1':1})
    #df['own_telephone']=df['own_telephone'].map({'yes':1, 'none':0})
    #df['foreign_worker']=df['foreign_worker'].map({'yes':1, 'no':0})


    # Casting to float for later purpose
    #print(df.head())
    #df = df['checking_status'].astype(float)
    #df = df.astype(float)
   # return df, target, feature_names


def get_df2(ds):
  df_new = pd.read_csv("/content/drive/MyDrive/transactions.csv")
  df_new.drop(['merchantCity','merchantState','merchantZip','echoBuffer','posOnPremises','recurringAuthInd', 'Unnamed: 0'],axis=1,inplace=True)
  df_new.drop(['accountNumber','customerId','merchantName'],axis=1,inplace=True)
  df_new['transactionDateTime'] = pd.to_datetime(df_new['transactionDateTime'])

  df_new['transactionDateTime_month'] = df_new['transactionDateTime'].dt.month
  df_new['transactionDateTime_hour'] = df_new['transactionDateTime'].dt.hour

  df_new.drop('transactionDateTime',axis = 1,inplace = True)
  df_new.drop('currentExpDate',axis = 1,inplace = True)
  df_new.drop('accountOpenDate',axis = 1,inplace = True)
  df_new.drop('dateOfLastAddressChange',axis = 1,inplace = True)


  df_new['availableMoney'] = df_new['availableMoney']/df_new['creditLimit']
  df_new['transactionAmount']=df_new['transactionAmount']/df_new['creditLimit']
  df_new['currentBalance']=df_new['currentBalance']/df_new['creditLimit']
  df_new['isCVVcorrect']=(df_new['cardCVV']==df_new['enteredCVV'])
  df_new['isSameCountry']=(df_new['acqCountry']==df_new['merchantCountryCode'])

  from sklearn.preprocessing import LabelEncoder
  le = LabelEncoder()
  var = ['posEntryMode','posConditionCode','merchantCategoryCode','transactionType','cardPresent','expirationDateKeyInMatch','isFraud']
  for i in var:
      df_new[i] = le.fit_transform(df_new[i])

  df_new.drop(['enteredCVV','cardCVV','merchantCountryCode','acqCountry','cardLast4Digits','creditLimit'],axis=1,inplace=True)
  X=df_new['isFraud'].copy()
  df_new.drop(['isFraud'],inplace=True,axis=1)
  df_new['isFraud'] = X

  feature_names = ['availableMoney', 'transactionAmount', 'posEntryMode',
       'posConditionCode', 'merchantCategoryCode', 'transactionType',
       'currentBalance', 'cardPresent', 'expirationDateKeyInMatch',
       'transactionDateTime_month','transactionDateTime_hour', 'isCVVcorrect', 'isSameCountry']
  target = 'isFraud'
  
  return df_new, target, feature_names

   

In [ ]:
def normalize(df, target, feature_names, bounds):
    df_return = df.copy()
    # Makes sure target does not need scaling
    targets = np.unique(df[target].values)
    assert(len(targets == 2) and 0. in targets and 1. in targets)
    
    scaler = MinMaxScaler()
    X = df_return[feature_names]
    scaler.fit(X)    
    df_return[feature_names] = scaler.transform(X)
    
    lower_bounds = scaler.transform([bounds[0]])
    upper_bounds = scaler.transform([bounds[1]])

    return scaler, df_return, (lower_bounds[0], upper_bounds[0])

def get_weights(df, target, show_heatmap=False):
    def heatmap(cor):
        plt.figure(figsize=(8,6))
        sns.heatmap(cor, annot=True, cmap=plt.cm.Reds)
        plt.show()

    cor = df.corr()
    cor_target = abs(cor[target])

    weights = cor_target[:-1] #removing target WARNING ASSUMES TARGET IS LAST
    weights = weights / np.linalg.norm(weights)

    if show_heatmap:
        heatmap(cor)
            
    return weights.values

def balance_df(df):
    len_df_0, len_df_1 = len(df[df[target] == 0.]), len(df[df[target] == 1.])
    df_0 = df[df[target] == 0.].sample(min(len_df_0, len_df_1), random_state=SEED)
    df_1 = df[df[target] == 1.].sample(min(len_df_0, len_df_1), random_state=SEED)
    df = pd.concat((df_0, df_1))
    return df

def get_bounds():
    low_bounds = df_orig.min().values
    up_bounds = df_orig.max().values
    
    #removing target WARNING ASSUMES TARGET IS LAST
    low_bounds = low_bounds[:-1]
    up_bounds = up_bounds[:-1]
    
    return [low_bounds, up_bounds]

def split_train_test_valid(val_size=50,test_size=300):
    # Train test splits
    print(df.shape)
    df_train, df_test = train_test_split(df, test_size=test_size, shuffle=True, random_state=SEED)
    df_test, df_valid = train_test_split(df_test, test_size=val_size, shuffle=True, random_state=SEED)
    
    return df_train, df_test, df_valid

In [ ]:
def get_model(conf, load=False):
    assert(conf['Dataset'] == 'credit-g')
    
    class Linear(nn.Module):
        def __init__(self, D_in, H, D_out):
            super(Linear, self).__init__()
            self.linear1 = torch.nn.Linear(D_in, H)
            self.linear2 = torch.nn.Linear(H, H)
            self.linear3 = torch.nn.Linear(H, D_out)
            self.relu = torch.nn.ReLU()
            self.softmax = torch.nn.Softmax(dim=0)

        def forward(self, x):
            h1 = self.relu(self.linear1(x))
            h2 = self.relu(self.linear2(h1))
            h3 = self.relu(self.linear2(h2))
            h4 = self.relu(self.linear2(h3))
            h5 = self.relu(self.linear2(h4))
            h6 = self.relu(self.linear2(h5))
            a3 = self.linear3(h6)
            y = self.softmax(a3)
            return y

    def train(model, criterion, optimizer, X, y, N, n_classes):
        model.train()

        current_loss = 0
        current_correct = 0


        # Training in batches
        for ind in range(0, X.size(0), N):
            indices = range(ind, min(ind + N, X.size(0)) - 1) 
            inputs, labels = X[indices], y[indices]
            inputs = Variable(inputs, requires_grad=True)


            optimizer.zero_grad()

            output = model(inputs)
            _, indices = torch.max(output, 1) # argmax of output [[0.61, 0.12]] -> [0]
            # [[0, 1, 1, 0, 1, 0, 0]] -> [[1, 0], [0, 1], [0, 1], [1, 0], [0, 1], [1, 0], [1, 0]]
            preds = torch.tensor(keras.utils.to_categorical(indices, num_classes=n_classes))

            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()

            current_loss += loss.item()
            current_correct += (preds.int() == labels.int()).sum() /n_classes


        current_loss = current_loss / X.size(0)
        current_correct = current_correct.double() / X.size(0)    

        return preds, current_loss, current_correct.item()
    
    df = conf['TrainData']
    target = conf['Target']
    feature_names = conf['FeatureNames']
                        
    n_classes = len(np.unique(df[target]))
    X_train = torch.FloatTensor(df[feature_names].values)
    y_train = keras.utils.to_categorical(df[target], n_classes)
    y_train = torch.FloatTensor(y_train)

    D_in = X_train.size(1)
    D_out = y_train.size(1)

    epochs = 400
    batch_size = 100
    H = 100
    net = Linear(D_in, H, D_out)

    lr = 1e-4    
    criterion = torch.nn.BCELoss()
    optimizer = torch.optim.Adam(net.parameters(), lr=lr)

    for epoch in range(epochs):
        preds, epoch_loss, epoch_acc = train(net, criterion, optimizer, X_train, y_train, batch_size, n_classes)     
        if (epoch % 50 == 0):
            print("> epoch {:.0f}\tLoss {:.5f}\tAcc {:.5f}".format(epoch, epoch_loss, epoch_acc))

    net.eval()
    
    return net

In [ ]:
def gen_adv(config, method):
    df_test = config['TestData']
    extra_cols = ['orig_pred', 'adv_pred', 'iters']    
    model = config['Model']
    weights = config['Weights']
    bounds = config['Bounds']
    maxiters = config['MaxIters']
    alpha = config['Alpha']
    lambda_ = config['Lambda']
    
    results = np.zeros((len(df_test), len(feature_names) + len(extra_cols)))    
            
    i = -1
    for _, row in tqdm_notebook(df_test.iterrows(), total=df_test.shape[0], desc="{}".format(method)):
        i += 1
        x_tensor = torch.FloatTensor(row[config['FeatureNames']])   
        
        if method == 'LowProFool':
            orig_pred, adv_pred, x_adv, loop_i = lowProFool(x_tensor, model, weights, bounds,
                                                             maxiters, alpha, lambda_)
            if(i==1):
              print(x_tensor, x_adv)

        elif method == 'Deepfool':
            orig_pred, adv_pred, x_adv, loop_i = deepfool(x_tensor, model, maxiters, alpha,
                                                          bounds, weights=[])
        else:
            raise Exception("Invalid method", method)
        results[i] = np.concatenate((x_adv, [orig_pred, adv_pred, loop_i]), axis=0)
        
    return pd.DataFrame(results, index=df_test.index, columns = feature_names + extra_cols)

In [ ]:
# Load initial dataset
df_orig, target, feature_names = get_df2(DATASET)
print(df_orig.columns)

# Balance dataset classes
df = balance_df(df_orig)

# Compute the bounds for clipping
bounds = get_bounds()

# Normalize the data
scaler, df, bounds = normalize(df, target, feature_names, bounds)

# Compute the weihts modelizing the expert's knowledge
weights = get_weights(df, target)

# Split df into train/test/valid
splits=int(df.shape[0]*0.2)
df_train, df_test, df_valid = split_train_test_valid(int(splits/2),splits)

# Build experimenation config
config = {'Dataset'     : 'credit-g',
         'MaxIters'     : 20000,
         'Alpha'        : 0.001,
         'Lambda'       : 8.5,
         'TrainData'    : df_train,
         'TestData'     : df_test,
         'ValidData'    : df_valid,
         'Scaler'       : scaler,
         'FeatureNames' : feature_names,
         'Target'       : target,
         'Weights'      : weights,
         'Bounds'       : bounds}

# Train neural network
model = get_model(config)
config['Model'] = model

# Compute accuracy on test set
y_true = df_test[target]
x_test = torch.FloatTensor(df_test[feature_names].values)
y_pred = model(x_test)
y_pred = np.argmax(y_pred.detach().numpy(), axis=1)
print("Accuracy score on test data", accuracy_score(y_true, y_pred))
    
# Sub sample
config['TestData'] = config['TestData'].sample(n=150, random_state = SEED)



Index(['availableMoney', 'transactionAmount', 'posEntryMode',
       'posConditionCode', 'merchantCategoryCode', 'transactionType',
       'currentBalance', 'cardPresent', 'expirationDateKeyInMatch',
       'transactionDateTime_month', 'transactionDateTime_hour', 'isCVVcorrect',
       'isSameCountry', 'isFraud'],
      dtype='object')


/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


(24834, 14)
> epoch 0	Loss 0.02303	Acc 0.57620
> epoch 50	Loss 0.02272	Acc 0.64959
> epoch 100	Loss 0.02267	Acc 0.65397
> epoch 150	Loss 0.02263	Acc 0.66061
> epoch 200	Loss 0.02260	Acc 0.66554
> epoch 250	Loss 0.02258	Acc 0.67012
> epoch 300	Loss 0.02255	Acc 0.67435
> epoch 350	Loss 0.02253	Acc 0.67636
Accuracy score on test data 0.6407571486105518


In [ ]:
def gen_adv2(config, method, cat_indices,max_vals,is_editable):
    df_test = config['TestData']
    extra_cols = ['orig_pred', 'adv_pred', 'iters']    
    model = config['Model']
    weights = config['Weights']
    bounds = config['Bounds']
    maxiters = config['MaxIters']
    alpha = config['Alpha']
    lambda_ = config['Lambda']
    
    results = np.zeros((len(df_test), len(feature_names) + len(extra_cols)))    
            
    i = -1
    for _, row in tqdm_notebook(df_test.iterrows(), total=df_test.shape[0], desc="{}".format(method)):
        i += 1
        x_tensor = torch.FloatTensor(row[config['FeatureNames']])   
        
        if method == 'LowProFool':
            orig_pred, adv_pred, x_adv, loop_i = lowProFool2(x_tensor, model, weights, bounds,
                                                             maxiters, alpha, lambda_,
                                                             cat_indices,max_vals,is_editable)
            if(i==1):
              print(x_tensor, x_adv)

        elif method == 'Deepfool':
            orig_pred, adv_pred, x_adv, loop_i = deepfool(x_tensor, model, maxiters, alpha,
                                                          bounds, weights=[])
        else:
            raise Exception("Invalid method", method)
        results[i] = np.concatenate((x_adv, [orig_pred, adv_pred, loop_i]), axis=0)
        
    return pd.DataFrame(results, index=df_test.index, columns = feature_names + extra_cols)

def lowProFool2(x, model, weights, bounds, maxiters, alpha, lambda_, cat_indices,max_vals, is_editable):
    """
    Generates an adversarial examples x' from an original sample x

    :param x: tabular sample
    :param model: neural network
    :param weights: feature importance vector associated with the dataset at hand
    :param bounds: bounds of the datasets with respect to each feature
    :param maxiters: maximum number of iterations ran to generate the adversarial examples
    :param alpha: scaling factor used to control the growth of the perturbation
    :param lambda_: trade off factor between fooling the classifier and generating imperceptible adversarial example
    :return: original label prediction, final label prediction, adversarial examples x', iteration at which the class changed
    """

    r = Variable(torch.FloatTensor(1e-8 * np.ones(x.numpy().shape)), requires_grad=True) 
    v = torch.FloatTensor(np.array(weights))
    num_idx = cat_indices
    n_print_samples=5
    n_print_counter=0
    
    output = model.forward(x + r)
    orig_pred = output.max(0, keepdim=True)[1].cpu().numpy()
    target_pred = np.abs(1 - orig_pred)
    
    target = [0., 1.] if target_pred == 1 else [1., 0.]
    target = Variable(torch.tensor(target, requires_grad=False)) 
    
    lambda_ = torch.tensor([lambda_])
    
    bce = nn.BCELoss()
    l1 = lambda v, r: torch.sum(torch.abs(v * r)) #L1 norm
    l2 = lambda v, r: torch.sqrt(torch.sum(torch.mul(v * r,v * r))) #L2 norm
    
    best_norm_weighted = np.inf
    best_pert_x = x
    
    loop_i, loop_change_class = 0, 0
    while loop_i < maxiters:
            
        zero_gradients(r)

        # Computing loss 
        loss_1 = bce(output, target)
        loss_2 = l2(v, r)
        loss = (loss_1 + lambda_ * loss_2)

        # Get the gradient
        loss.backward(retain_graph=True)
        grad_r = r.grad.data.cpu().numpy().copy()
        
        # Guide perturbation to the negative of the gradient
        ri = - grad_r
    
        # limit huge step
        ri *= alpha

        # Adds new perturbation to total perturbation
        r = r.clone().detach().cpu().numpy() + ri
        
        # For later computation
        r_norm_weighted = np.sum(np.abs(r * weights))
        
        # Ready to feed the model
        r = Variable(torch.FloatTensor(r), requires_grad=True) 
        
        # Compute adversarial example
        xprime = x + r
        
        # Clip to stay in legitimate bounds
        xprime = clip(xprime, bounds[0], bounds[1])
        # print(xprime)
        # gfun = xprime.grad_fn
        
        counter = 0

        # if n_print_counter<n_print_samples:
        #     print("before mod")
        #     print(xprime,x)

        for cat_idx, bnd, editable in zip( cat_indices, max_vals, is_editable):
          # print(x.item())
          # print(bnd)
          if editable:
            if cat_idx:
              if xprime[counter].item()==0:
                xprime[counter] = 0
              else:

                # if n_print_counter<n_print_samples:
                #   print(counter, float(round(xprime[counter].item()*bnd)/bnd))

                xprime[counter] = float(round(xprime[counter].item()*bnd)/bnd)
          else:
            xprime[counter]  = x[counter]
          counter += 1
          # print(counter)
          # print(xprime)

        # if n_print_counter<n_print_samples:
        #   print("after mod")
        #   print(xprime)
        #   n_print_counter+=1
            

        
        # print(xprime)
        
        # Classify adversarial example
        output = model.forward(xprime)
        output_pred = output.max(0, keepdim=True)[1].cpu().numpy()
        
        # Keep the best adverse at each iterations
        if output_pred != orig_pred and r_norm_weighted < best_norm_weighted:
            best_norm_weighted = r_norm_weighted
            best_pert_x = xprime

        if output_pred == orig_pred:
            loop_change_class += 1
            
        loop_i += 1 
        
    # Clip at the end no matter what
    best_pert_x = clip(best_pert_x, bounds[0], bounds[1])
    output = model.forward(best_pert_x)
    output_pred = output.max(0, keepdim=True)[1].cpu().numpy()

    return orig_pred, output_pred, best_pert_x.clone().detach().cpu().numpy(), loop_change_class 


In [ ]:
from copy import deepcopy
config2 = deepcopy(config)
config2['TestData'] = config2['TestData'].sample(n=150, random_state = SEED)
max_vals = df_orig.max().astype(float).tolist()[:-1]
is_editable = [False, True, False, False, True, False, False, False, False, True, True, True , True]

lpf_test = gen_adv2(config2,"LowProFool",
                    cat_indices=[False, False, True, True, True, True, False, True, True, True, True, True, True],
                    max_vals=max_vals, is_editable=is_editable )

<ipython-input-15-82ebe7c618f2>:14: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for _, row in tqdm_notebook(df_test.iterrows(), total=df_test.shape[0], desc="{}".format(method)):


LowProFool:   0%|          | 0/150 [00:00<?, ?it/s]

<__array_function__ internals>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


tensor([0.8284, 0.0199, 0.4000, 0.0000, 0.7222, 0.3333, 0.1716, 0.0000, 0.0000,
        0.1818, 0.2174, 1.0000, 1.0000]) [0.82844895 0.01988091 0.4        0.         0.7222222  0.33333334
 0.17155103 0.         0.         0.18181819 0.2173913  1.
 1.        ]


In [ ]:
lpf_test['adv_pred'].value_counts()


1.0    121
0.0     29
Name: adv_pred, dtype: int64

In [ ]:
lpf_test['orig_pred'].value_counts()

1.0    99
0.0    51
Name: orig_pred, dtype: int64

In [ ]:
df_test.head()

,availableMoney,transactionAmount,posEntryMode,posConditionCode,merchantCategoryCode,transactionType,currentBalance,cardPresent,expirationDateKeyInMatch,transactionDateTime_month,transactionDateTime_hour,isCVVcorrect,isSameCountry,isFraud
493988,0.905367,0.392517,0.2,0.333333,0.777778,0.333333,0.094633,0.0,0.0,0.181818,0.913043,1.0,1.0,1
612649,0.833570,0.217468,0.4,0.000000,0.722222,0.333333,0.166430,0.0,0.0,0.818182,0.260870,1.0,1.0,1
376099,0.976335,0.011791,0.4,0.000000,0.277778,0.666667,0.023665,1.0,0.0,0.000000,0.739130,1.0,1.0,1
688010,0.949952,0.003790,0.2,0.000000,0.277778,0.333333,0.050048,1.0,0.0,0.000000,0.695652,1.0,1.0,1
155150,0.828919,0.000829,0.2,0.000000,0.777778,0.333333,0.171081,0.0,0.0,1.000000,0.869565,1.0,1.0,1


In [ ]:
df_test['posEntryMode'].value_counts()

0.4    907
0.2    771
0.0    651
0.8     65
0.6     60
1.0     29
Name: posEntryMode, dtype: int64

In [ ]:
lpf_test['posEntryMode'].value_counts()

0.4    58
0.2    41
0.0    40
0.6     6
0.8     3
1.0     2
Name: posEntryMode, dtype: int64

In [ ]:
lpf_test['posConditionCode'].unique()

array([0.        , 0.66666669, 0.33333334])

In [ ]:
df_test['posConditionCode'].value_counts()

0.000000    1987
0.333333     454
0.666667      39
1.000000       3
Name: posConditionCode, dtype: int64

In [ ]:
df_test.shape

(2483, 14)

In [ ]:
lpf_test['cardPresent'].unique()

array([0., 1.])

In [ ]:
df_test['cardPresent'].unique()

array([0., 1.])

In [ ]:
lpf_test['isCVVcorrect'].unique()

array([1., 0.])

In [ ]:
df_test['isCVVcorrect'].unique()

array([1., 0.])

In [ ]:
df_test['isSameCountry'].unique()

array([1., 0.])

In [ ]:
lpf_test['isSameCountry'].unique()

array([1.])